# 問題
それぞれについてJupyter Notebookにマークダウン形式で記述してください。


(1) 物体検出の分野にはどういった手法が存在したか。


(2) Fasterとあるが、どういった仕組みで高速化したのか。


(3) One-Stageの手法とTwo-Stageの手法はどう違うのか。


(4) RPNとは何か。


(5) RoIプーリングとは何か。


(6) Anchorのサイズはどうするのが適切か。


(7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。


(8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

## 条件
* 答える際は論文のどの部分からそれが分かるかを書く。
* 必要に応じて先行研究（引用されている論文）も探しにいく。最低2つは他の論文を利用して回答すること。
* 論文の紹介記事を見ても良い。ただし、答えは論文内に根拠を探すこと。

### (1) 物体検出の分野にはどういった手法が存在したか。

物体検出を行うモデルにはR-CNN(Regional)やSPPnet(Spatial Pyramid Pooling Networks)など、領域候補（Region Proposal）の抽出する手法が採用されている。
>(Recent advances in object detection are driven by the success of region proposal methods (e.g., [4]) and region-based convolutional neural networks (R- CNNs) [5]) 引用元 --R. Girshick, “Fast R-CNN,” in IEEE International Conference on Computer Vision (ICCV), 2015


### (2) Fasterとあるが、どういった仕組みで高速化したのか。

領域検出を行うFast-CNNなどのDeep-CNNから計算されるconvolutionsを物体検出に使用されるモデルと共有することによって、計算時間を短縮することに成功した。
>引用元：p1 we introduce novel Region Proposal Networks (RPNs) that share convolutional layers with state-of-the-art object detection networks [1], [2]. By sharing convolutions at test-time, the marginal cost for computing proposals is small (e.g., 10ms per image).

また、検出の際には、既存のマルチスケーリングや多数のフィルタサイズを使用するのではなく、anchor boxesを採用することによって、多くのサンプルやフィルタをiterateすることを回避でき、高速化につながった。
>引用元：p1-p2 In contrast to prevalent methods [8], [9], [1], [2] that use pyramids of images (Figure 1, a) or pyramids of filters (Figure 1, b), we introduce novel “anchor” boxes that serve as references at multiple scales and aspect ratios.



### (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

One-stage Detection手法とは、feature map上をフィルタで探索し、領域候補の抽出及び検出物体の分類を同時に行う手法である。他方、Two-stage Proposalとは領域候補と分類を直列に連結された個別のモデルが行う手法である。両手法共スライド式のフィルタを用いているが、one-stageでは前述の通りフィルタの出力featuresで物体の位置及び分類を行う。Two-stageでのフィルタの役割はあくまで領域候補の抽出のみで、上位のモデル（Fast RNN）が出力された領域候補の精査・分類を行う。両手法の性能を比較したところ、two-stageはone-stageより約4.8%の精度向上を示し、計算時間もより早い。
> 引用元：p10 (The OverFeat paper [9] proposes a detection method that uses regressors and classifiers on sliding windows over convolutional feature maps. OverFeat is a one-stage, class-specific detection pipeline, and ours is a two-stage cascade consisting of class-agnostic pro- posals and class-specific detections.)

# (4) RPNとは何か。

Region Proposal Networkのこと。
RPNは、各位置における物体境界と物体性スコアを同時に予測する完全畳み込みネットワークです。

領域提案ネットワーク(RPN)は，(任意のサイズの)画像を入力として受け取り，それぞれがオブジェクトネススコアを持つ長方形のオブジェクト提案のセットを出力します。
我々の最終的な目標は、高速R-CNN物体検出ネットワークと計算を共有することであるため、両方のネットが共通の畳み込み層のセットを共有していると仮定しています。

>* 引用元：P1 Abstract : An RPN is a fully convolutional network that simultaneously predicts object bounds and objectness scores at each position. 

>* 引用元 : P3 Region Proposal Network : A Region Proposal Network (RPN) takes an image(of any size) as input and outputs a set of rectangular　object proposals, each with an objectness score.
3 We　model this process with a fully convolutional network[7], which we describe in this section. Because our ultimate goal is to share　computation with a Fast R-CNN　object detection network [2], we assume that both nets　share a common set of convolutional layers. 
In our experiments, we investigate the Zeiler and Fergus model[32] (ZF), which has 5 shareable convolutional layers　and the Simonyan and Zisserman model [3] (VGG-16),　which has 13 shareable convolutional layers.

### (5) RoIプーリングとは何か。

RoI Poolingとは、Fast R-CNNで実装された機構であり、抽出されたn個のRoI (Region of Interest)を固定されたサイズHxWのfeature mapにMaxPoolingされる。
> 引用元：R. Girshick, Fast R-CNN p1より(The RoI pooling layer uses max pooling to convert the features inside any valid region of interest into a small feature map with a fixed spatial extent of H × W (e.g., 7 × 7), where H and W are layer hyper-parameters that are inde- pendent of any particular RoI)



![RoIPooling animation](https://raw.githubusercontent.com/deepsense-ai/roi-pooling/master/roi_pooling_animation.gif "RoI Pooling Image animation")

Faster R-CNNでは、Fast版と違ってRoIが固定の大きさなため、プーリング後のfeature map上でも全RoIが同サイズである。異なるRoIサイズに対応するため、RoIサイズひとつ毎にfeature mapを生成し、ひとつのfeature mapにつき回帰モジュール (regressors)を別に用意する。この手法によって、異なるRoIサイズ間にて重みが共有されることを回避できる。
> 引用元：p5 (In our formula- tion, the features used for regression are of the same spatial size (3 × 3) on the feature maps. To account for varying sizes, a set of k bounding-box regressors are learned. Each regressor is responsible for one scale and one aspect ratio, and the k regressors do not share weights.)


### (6) Anchorのサイズはどうするのが適切か。

3つのanchor size（$128^2$, $256^2$, $512^2$）と１つまたは３つのアスペクト比（2:1, 1:1, 1:2）が採用データセット（PASCAL VOC 2007）に対して最もmAP値が高く、複数のサイズのアンカーを回帰参照として使用することが効果的な解決策であることを示しています。
アスペクト比の使用数はmAPに影響がなく、anchor sizeとアスペクト比に相関があると考察されていますが、システムの柔軟性を保つためにこの2つの次元を採用しています。

> 引用元：p6 : For anchors, we use 3 scales with box areas of 1282, 2562, and 5122 pixels, and 3 aspect ratios of 1:1, 1:2, and 2:1. These hyper-parameters are not carefully cho- sen for a particular dataset, and we provide ablation experiments on their effects in the next section.

> 引用元: p9 : The mAP is higher if using 3 scales (with 1 aspect ratio) or 3 aspect ratios (with 1 scale), demonstrating that using anchors of multiple sizes as the regression references is an effective solution. Using just 3 scales with 1 aspect ratio (69.8%) is as good as using 3 scales with 3 aspect ratios on this dataset, suggesting that scales and aspect ratios are not disentangled dimensions for the detection accuracy. But we still adopt these two dimensions in our designs to keep our system flexible.

### (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

本著ではPASCAL VOC 2007＋2012、及びMicrosoft COCOのデータセットが使用されています。このデータセットは、約5k枚のtrainval画像と約5k枚のテスト画像から構成されており、20のオブジェクトカテゴリにまたがっています。

> 引用元：p2 :We comprehensively evaluate our method on the PASCAL VOC 2007 detection benchmark [11]. This dataset consists of about 5k trainval images and 5k test images over 20 object categories.


このデータセットには80個のオブジェクトカテゴリが含まれている。このデータセットでは、トレーニングセットに80k枚、検証セットに40k枚、テストデバ イスに20k枚の画像を用いて実験を行っている。IoU ∈[0.5 : 0.05 : 0.95] (COCOの標準的なメトリック、単にmAP@[.5, .95]と表記)と mAP@0.5 (PASCAL VOCのメトリック)の平均値であるmAPを評価する。

> 引用元：p10 : This dataset involves 80 object categories. We experiment with the 80k images on the training set, 40k images on the validation set, and 20k images on the test-dev set. We evaluate the mAP averaged for IoU ∈ [0.5 : 0.05 : 0.95] (COCO’s standard metric, simply denoted as mAP@[.5, .95]) and mAP@0.5 (PASCAL VOC’s metric)


先行研究との比較にはCOCOデータセットにて検証しており、testデータに対してFast R-CNNの評価がmAP@0.5: 39.3%とmAP@[.5, .95]: 19.3%だったところ、Faster R-CNNだとmAP@0.5: 42.1%とmAP@[.5, .95]: 21.5%でした。これは、同じプロトコルでの高速RCNNと比較して、mAP@0.5 が 2.8%、mAP@[.5, .95] が 2.2%高くなっています
このことから、RPN は、より高い IoU しきい値での定位精度の向上に優れていることがわかります。

> 引用元:P11 : In Table 11 we first report the results of the Fast R-CNN system [2] using the implementation in this paper. Our Fast R-CNN baseline has 39.3% mAP@0.5 on the test-dev set, higher than that reported in [2].
We conjecture that the reason for this gap is mainly due to the definition of the negative samples and also the changes of the mini-batch sizes. We also note that
the mAP@[.5, .95] is just comparable. Next we evaluate our Faster R-CNN system. Using the COCO training set to train, Faster R-CNN has 42.1% mAP@0.5 and 21.5% mAP@[.5, .95] on the COCO test-dev set. This is 2.8% higher for mAP@0.5 and 2.2% higher for mAP@[.5, .95] than the Fast RCNN counterpart under the same protocol (Table 11).
This indicates that RPN performs excellent for improving the localization accuracy at higher IoU thresholds. Using the COCO trainval set to train, Faster RCNN has 42.7% mAP@0.5 and 21.9% mAP@[.5, .95] on the COCO test-dev set. Figure 6 shows some results on the MS COCO test-dev set.

### (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。